# Comfy Kettenregel (Autograd DIY) - univariate, skalare Funktionen

$$F(x) = f_1 \circ f_2 = f_1(f_2(x)) \Rightarrow f_1'(f_2(x)) \cdot f'_2(x)$$

$$F(x) = f_1 \circ f_2 \circ f_3 = f_1(f_2(f_3(x))) \Rightarrow f_1'(f_2(f_3(x))) \cdot f_2'(f_3
(x)) \cdot f_3'(x)$$

## Aufgabe

Ziel: Gradientenbasierte Optimierung von $f(x) = \sqrt{\frac{1}{e^{\sin(x)}}}$


In [ ]:
import math


def one_div_x(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = 1 / x
    derivative = -inner_derivative / x**2

    return value, derivative


def sin(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sin(x)
    derivative = math.cos(x) * inner_derivative

    return value, derivative


def sqrt(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.sqrt(x)
    derivative = 1 / (2 * math.sqrt(x)) * inner_derivative

    return value, derivative


def exp(x: float, inner_derivative: float = 1) -> tuple[float, float]:

    value = math.exp(x)
    derivative = math.exp(x) * inner_derivative

    return value, derivative

In [ ]:
def f_x(x: float) -> tuple[float, float]:
    return sqrt(*one_div_x(*exp(*sin(x))))

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

x = np.arange(-5, 5, 0.01)

res = [f_x(_) for _ in x]
y, derivative = zip(*res)

df = pd.DataFrame(
    {
        "x": x,
        "y": y,
        "derivative": derivative,
    }
)

px.line(df, x="x", y="y")

In [ ]:
x_start = 4.0
x_min = x_start - 8.0
x_max = x_start + 8.0
xs = []
ys = []

lr = 1e-2
significant_gradient = 1e-3
iter = 1

while True:
    y, deriv = f_x(x_start)
    if np.fabs(deriv) >= significant_gradient:
        xs.append(x_start)
        ys.append(y)
        x_start -= lr * deriv
        print(iter, x_start, y) if iter % 100 == 0 or iter == 1 else None
    else:
        xs.append(x_start)
        ys.append(y)
        break
    iter += 1

In [ ]:
import plotly.graph_objects as go

x = np.arange(x_min, x_max, 0.01)

res = [f_x(_) for _ in x]
y, _ = zip(*res)

fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(color="green", width=1),
            name="Function Graph",
        ),
        go.Scatter(
            x=[xs[0]],
            y=[ys[0]],
            mode="markers",
            marker=dict(color="red", size=10),
            name="Current Position",
        ),
    ]
)

# update layout parameters and add start button for animation
fig.update_layout(
    width=1400,
    height=900,
    xaxis=dict(range=(np.min(x) - 1, np.max(x) + 1), autorange=False),
    yaxis=dict(range=(np.min(y) - 1, np.max(y) + 1), autorange=False),
    title_text="Gradient Descent Animation",
    # start button config
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 5, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 0, "easing": "linear"},
                        },
                    ],
                    label="start",
                    method="animate",
                )
            ],
        )
    ],
)

# specify the animation frames
fig.update(
    frames=[
        go.Frame(data=[go.Scatter(x=[xs[k]], y=[ys[k]])], traces=[1])
        for k in range(len(ys))
    ]
)

# show result
fig.show()